In [1]:
library('fastDummies')
library('pmml')
library('missForest')

Loading required package: XML

Loading required package: randomForest

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.

Loading required package: foreach

Loading required package: itertools

Loading required package: iterators



In [2]:
data <- read.csv('./hmeq.csv', header=T, na.string=c(""))

In [3]:
ncol(data)

[1] 13

In [4]:
head(data)

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<dbl>
1,1,1100,25860,39025,HomeImp,Other,10.5,0,0,94.36667,1,9,NA
2,1,1300,70053,68400,HomeImp,Other,7.0,0,2,121.83333,0,14,NA
3,1,1500,13500,16700,HomeImp,Other,4.0,0,0,149.46667,1,10,NA
4,1,1500,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,0,1700,97800,112000,HomeImp,Office,3.0,0,0,93.33333,0,14,NA
6,1,1700,30548,40320,HomeImp,Other,9.0,0,0,101.46600,1,8,37.11361


In [5]:
data <- data.frame( CLAGE = as.numeric(data$CLAGE),
                 CLNO = as.numeric(data$CLNO),
                 DEBTINC = as.numeric(data$DEBTINC),
                 DELINQ = as.numeric(data$DELINQ),
                 DEROG = as.numeric(data$DEROG),
                 LOAN = as.numeric(data$LOAN),
                 MORTDUE = as.numeric(data$MORTDUE),
                 NINQ = as.numeric(data$NINQ),
                 VALUE = as.numeric(data$VALUE),
                 YOJ = as.numeric(data$YOJ),
                 JOB = as.factor(data$JOB),
                 REASON = as.factor(data$REASON),
                 BAD = as.factor(data$BAD),
                 stringsAsFactors = FALSE)

In [6]:
imp <- missForest(data, maxiter = 2, ntree = 10)

  missForest iteration 1 in progress...done!
  missForest iteration 2 in progress...done!


In [7]:
imp$OOBerror

NRMSE       PFC 
0.1643078 0.1052781

In [8]:
imp_f <- imp$ximp

In [10]:
data_dummies <- dummy_cols(imp_f)
head(data_dummies)

,CLAGE,CLNO,DEBTINC,DELINQ,DEROG,LOAN,MORTDUE,NINQ,VALUE,YOJ,...,JOB_Mgr,JOB_Office,JOB_Other,JOB_ProfExe,JOB_Sales,JOB_Self,REASON_DebtCon,REASON_HomeImp,BAD_0,BAD_1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,94.36667,9.0,31.12070,0.0,0.0,1100,25860.0,1.0,39025.0,10.50,...,0,0,1,0,0,0,0,1,0,1
2,121.83333,14.0,33.00099,2.0,0.0,1300,70053.0,0.0,68400.0,7.00,...,0,0,1,0,0,0,0,1,0,1
3,149.46667,10.0,23.46948,0.0,0.0,1500,13500.0,1.0,16700.0,4.00,...,0,0,1,0,0,0,0,1,0,1
4,150.62291,25.7,37.19249,2.8,1.1,1500,57196.8,1.7,64454.5,8.51,...,0,0,0,1,0,0,0,1,0,1
5,93.33333,14.0,30.19471,0.0,0.0,1700,97800.0,0.0,112000.0,3.00,...,0,1,0,0,0,0,0,1,1,0
6,101.46600,8.0,37.11361,0.0,0.0,1700,30548.0,1.0,40320.0,9.00,...,0,0,1,0,0,0,0,1,0,1


In [11]:
final_df <- within(data_dummies, rm("JOB", "REASON", "BAD_1", "BAD_0"))
final_df$BAD =data_dummies$BAD

In [12]:
head(final_df)

,CLAGE,CLNO,DEBTINC,DELINQ,DEROG,LOAN,MORTDUE,NINQ,VALUE,YOJ,BAD,JOB_Mgr,JOB_Office,JOB_Other,JOB_ProfExe,JOB_Sales,JOB_Self,REASON_DebtCon,REASON_HomeImp
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,94.36667,9.0,31.12070,0.0,0.0,1100,25860.0,1.0,39025.0,10.50,1,0,0,1,0,0,0,0,1
2,121.83333,14.0,33.00099,2.0,0.0,1300,70053.0,0.0,68400.0,7.00,1,0,0,1,0,0,0,0,1
3,149.46667,10.0,23.46948,0.0,0.0,1500,13500.0,1.0,16700.0,4.00,1,0,0,1,0,0,0,0,1
4,150.62291,25.7,37.19249,2.8,1.1,1500,57196.8,1.7,64454.5,8.51,1,0,0,0,1,0,0,0,1
5,93.33333,14.0,30.19471,0.0,0.0,1700,97800.0,0.0,112000.0,3.00,0,0,1,0,0,0,0,0,1
6,101.46600,8.0,37.11361,0.0,0.0,1700,30548.0,1.0,40320.0,9.00,1,0,0,1,0,0,0,0,1


In [13]:
model <- glm(formula = BAD ~ ., family = binomial(link = "logit"), data = final_df)

s <- summary(model)

In [14]:
s


Call:
glm(formula = BAD ~ ., family = binomial(link = "logit"), data = final_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2745  -0.5958  -0.4065  -0.2186   3.4772  

Coefficients: (2 not defined because of singularities)
                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -3.353e+00  3.104e-01 -10.801  < 2e-16 ***
CLAGE          -2.995e-03  5.017e-04  -5.970 2.38e-09 ***
CLNO           -2.946e-02  4.496e-03  -6.553 5.64e-11 ***
DEBTINC         1.009e-01  6.247e-03  16.148  < 2e-16 ***
DELINQ          7.449e-01  3.912e-02  19.040  < 2e-16 ***
DEROG           5.191e-01  4.769e-02  10.886  < 2e-16 ***
LOAN           -2.477e-05  4.376e-06  -5.662 1.50e-08 ***
MORTDUE        -7.882e-06  1.837e-06  -4.290 1.79e-05 ***
NINQ            1.420e-01  2.096e-02   6.778 1.22e-11 ***
VALUE           5.123e-06  1.316e-06   3.893 9.90e-05 ***
YOJ            -1.311e-02  5.745e-03  -2.282  0.02247 *  
JOB_Mgr        -6.017e-01  2.311e-01  -2.604  0.0092

In [15]:
print(s)


Call:
glm(formula = BAD ~ ., family = binomial(link = "logit"), data = final_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2745  -0.5958  -0.4065  -0.2186   3.4772  

Coefficients: (2 not defined because of singularities)
                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -3.353e+00  3.104e-01 -10.801  < 2e-16 ***
CLAGE          -2.995e-03  5.017e-04  -5.970 2.38e-09 ***
CLNO           -2.946e-02  4.496e-03  -6.553 5.64e-11 ***
DEBTINC         1.009e-01  6.247e-03  16.148  < 2e-16 ***
DELINQ          7.449e-01  3.912e-02  19.040  < 2e-16 ***
DEROG           5.191e-01  4.769e-02  10.886  < 2e-16 ***
LOAN           -2.477e-05  4.376e-06  -5.662 1.50e-08 ***
MORTDUE        -7.882e-06  1.837e-06  -4.290 1.79e-05 ***
NINQ            1.420e-01  2.096e-02   6.778 1.22e-11 ***
VALUE           5.123e-06  1.316e-06   3.893 9.90e-05 ***
YOJ            -1.311e-02  5.745e-03  -2.282  0.02247 *  
JOB_Mgr        -6.017e-01  2.311e-01  -2.604  0.0092

In [16]:
saveXML(pmml(model, model.name="General_Regression_Model",
             app.name="Rattle/PMML",
             description="Generalized Linear Regression Model"), "R_HMEQ.xml")

[1] "R_HMEQ.xml"

In [18]:
write.csv(final_df, './hmeq_dummies.csv', row.names=FALSE)